In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import regex as re
import time

In [2]:
def counter(df, label):
  word_count = {}
  for cat in categories:
    word_dict = {}
    data = df[df[label] == cat]['headline'] +' '+ df[df[label] == cat]['short_description']
    for row in data:
      words = list(set(re.split("[^a-zA-Z]",row)))#.split(' ')
      for word in words:
        word = word.lower()
        if word in word_dict:
          word_dict[word] = word_dict[word]+1
        else:
          word_dict[word] = 1
    for word, count in dict(word_dict).items():
      if (len(word)<3) or (count<4):
        word_dict.pop(word)
    word_count[cat] = word_dict
  return pd.DataFrame(word_count).fillna(0)

In [3]:
def prob_cond(word, category, word_df, a=0):
  """word: word whose probability needs to be found
     category: prior condition of the probabilty
     word_df: dictionary of the count of word in categories
     a: laplace smoothing factor"""
  word = word.lower()
  category = category.upper()
  if word in vocab:
    prob = (word_df.loc[word,category]+a)/(word_df[category].sum()+a*len(word_df[category]))
  else:
    prob = (a)/(word_df[category].sum()+a*len(word_df[category]))
  return prob

In [4]:
def prob(word, prob_df):
  word = word.lower()
  probblty = 0
  try:
    for cat in prob_df.columns:
      probblty += prob_df.loc[word,cat]*p_cat[cat]
    return probblty
  except:
    return probblty


In [5]:
def train(df,label='category', smoothing = 1):
  global categories
  categories = df[label].unique()
  temp = df[label].value_counts(sort=False)/df[label].value_counts().sum()
  p_cat = temp.to_dict()

  word_count = counter(df, label)
  global vocab
  vocab = word_count.index
  pc_word = {}
  for cat in categories:
    word_prob = {}
    for word in vocab:#word_count[cat].keys():
      word_prob[word] = prob_cond(word,cat,word_count,a=smoothing)
    pc_word[cat] = word_prob
  pc_word = pd.DataFrame(pc_word)
  return p_cat, word_count, pc_word

In [6]:
def predict(X):
  cat_pred = np.array([])
  #data = X#df_val['headline']#df_val['short_description']+' '+df_val['headline']
  for row in X:
    val_words = list(set(re.split("[^a-zA-Z]",row)))
    for word in val_words:
      if (len(word)<3):
        val_words.remove(word)
    cat_prob = np.array([])
    for cat in categories:
      list_prob = np.array([])
      for word in val_words:
        if word in vocab:#try:
          list_prob = np.append(list_prob,pc_word.loc[word,cat])#,word_count,a=1))
        else:
          list_prob = np.append(list_prob,prob_cond(word,cat,word_count,a=smoothing))
      cat_prob = np.append(cat_prob,np.prod(list_prob)*p_cat[cat])
    cat_prob = cat_prob/sum(cat_prob)
    # id = np.argmax(cat_prob)
    cat_pred = np.append(cat_pred,np.argmax(cat_prob))
  return cat_pred

In [7]:
def accuracy(prediction, actual):
  accu = 0
  for i in range(len(prediction)):
    if prediction[i] == actual[i]:
      accu += 1
  accu = accu/len(prediction)
  return accu

In [8]:
# !wget -O news.json 'https://drive.google.com/uc?export=download&id=1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp'
# # https://drive.google.com/file/d/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp/view?usp=sharing

In [9]:
df = pd.read_json('../input/news-category-dataset/News_Category_Dataset_v2.json',lines=True)
df.head()

In [10]:
len(df.category.unique())

In [11]:
df.category.unique()

In [12]:
df['category'] = df['category'].replace(['CULTURE & ARTS','THE WORLDPOST','PARENTS','ARTS','STYLE','TASTE','WELLNESS','WORLDPOST'],
                                        ['ARTS & CULTURE','WORLD NEWS','PARENTING','ARTS & CULTURE','STYLE & BEAUTY','FOOD & DRINK','HEALTHY LIVING','WORLD NEWS'])

In [13]:
len(df.category.unique())

In [14]:
df.category.unique()

In [15]:
df.info()

In [16]:
df, df_test = train_test_split(df, test_size=0.1, stratify=df['category'])
# df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)

In [17]:
smoothing = 1

In [18]:
# p_cat, word_count, pc_word = train(df, 'category', stpwrds=stpwrds, smoothing=smoothing)
# cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}


In [19]:
# start = time.time()
# input = df_val['headline']#+' '+df_val['short_description']
# predictions = predict(input)
# predictions = [cat_dict[v] for v in predictions]
# print(time.time()-start)

In [20]:
cv= 5
score = []
for i in range(cv):
    df_train, df_val = train_test_split(df, test_size=0.1)#, random_state=42)
    p_cat, word_count, pc_word = train(df_train, 'category', smoothing=smoothing)
    cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
    input = df_val['headline']#+' '+df_val['short_description']
    predictions = predict(input)
    predictions = [cat_dict[v] for v in predictions]
    actual = df_val['category'].values
    score.append(accuracy(predictions,actual))
np.mean(score)

In [21]:
score

In [22]:
p_cat, word_count, pc_word = train(df, 'category',smoothing=smoothing)
cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
print("Probability of 'The' occuring in a headline is: ", round(prob('The',pc_word),3))
print("Probability of 'The' occuring in a 'Business' headline is: ", round(prob_cond('The'.lower(),'BUSINESS'.upper(),word_count, a=smoothing),3))

In [23]:
input = df_test['headline']+' '+df_test['short_description']
predictions = predict(input)
predictions = [cat_dict[v] for v in predictions]
actual = df_test['category'].values
accuracy(predictions,actual)

In [24]:
full_prob = []
for cat in categories:
  word_prob = []
  for word in vocab:
    word_prob.append(pc_word.loc[word,cat]*p_cat[cat]/prob(word,pc_word))
  full_prob.append(word_prob)
full_prob = np.array(full_prob)
full_prob_df = pd.DataFrame(full_prob.T,columns=categories,index=vocab)

top_word = []
for cat in categories:
  top_word.append(full_prob_df[cat].sort_values(ascending=False).index)
top_word = np.array(top_word)
top_words_df = pd.DataFrame(top_word.T,columns=categories)

In [25]:
top_words_df.iloc[:10,:16]

In [26]:
top_words_df.iloc[:10,16:]